#Paquetes necesarios

In [1]:
import cv2  
import math
from ultralytics import YOLO

Modelos preentrenados, visualizando con las utilidades de ultralytics

In [2]:
# Carga del modelo
#model = YOLO('yolo11n.pt') #Contenedores
#model = YOLO('yolo11n-seg.pt') #Máscaras
model = YOLO('yolo11n-pose.pt')  #Pose

#Para un vídeo 
filename = "TGC23_PdH_C0056cut.mp4"
results = model(filename, show=True)

cv2.destroyAllWindows()


WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/375) c:\Users\kevin\Ingeniera informtica\Cuarto\VC\P4\git\otsedom.github.io\VC\P4\TGC23_PdH_C0056cut.mp4: 384x640 (no detections), 74.1ms
video 1/1 (frame 2/375) c:\Users\kevin\Ingeniera informtica\Cuarto\VC\P4\git\otsedom.github.io\VC\P4\TGC23_PdH_C0056cut.mp4: 384x640 (no detections), 58.0ms
video 1/1 (frame 3/375) c:\Users\kevin\Ingeniera informtica\Cuarto\VC\P4\git\otsedom.github.io\VC\P4\TGC23_PdH_C0056cut.mp4: 384x640 (no detectio

Desde cámara, detección con yolo11, modelo nano. Visualización propia con OpenCV

In [5]:
# Carga del modelo, descarga en disco si no está presente en la carpeta
model = YOLO('yolo11n.pt') #Contenedores

# Etiqueta de las distintas clases
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]


# Captura desde la webcam
vid = cv2.VideoCapture(0)
  
while(True):      
    # fotograma a fotograma
    ret, img = vid.read()
  
    # si hay imagen válida
    if ret:  
        # Detecta en la imagen
        results = model(img, stream=True)
        
        # Para cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
                
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100
                print("Confianza --->",confidence)

                # Clase
                cls = int(box.cls[0])
                print("Clase -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0

                # Dibuja el contenedor y clase
                cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(img, classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

        # Muestra fotograma
        cv2.imshow('Vid', img)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()


0: 480x640 1 person, 66.6ms
Confianza ---> 0.94
Clase --> person
Speed: 9.2ms preprocess, 66.6ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 77.1ms
Confianza ---> 0.94
Clase --> person
Speed: 1.2ms preprocess, 77.1ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 61.4ms
Confianza ---> 0.94
Clase --> person
Speed: 1.2ms preprocess, 61.4ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 63.1ms
Confianza ---> 0.94
Clase --> person
Speed: 1.9ms preprocess, 63.1ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 52.9ms
Confianza ---> 0.94
Clase --> person
Speed: 1.1ms preprocess, 52.9ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 88.0ms
Confianza ---> 0.94
Clase --> person
Speed: 2.3ms preprocess, 88.0ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 

Seguimiento. Requiere instalar lap con pip install lap

In [6]:
from collections import defaultdict
import numpy as np

# Carga del modelo, descarga en disco si no está presente en la carpeta
model = YOLO('yolo11n.pt') #Contenedores

# Etiqueta de las distintas clases
classNames = ["person", "bicycle", "car"]


# Captura desde la webcam
vid = cv2.VideoCapture(0)
track_history = defaultdict(lambda: [])
  
while(True):      
    # fotograma a fotograma
    ret, img = vid.read()
  
    # si hay imagen válida
    if ret:  
        # Seguimiento, con persistencia entre fotogramas
        results = model.track(img, persist=True, classes = [0,1,2])

        if 0:
            if results is not None:
                print(results[0])
                boxes = results[0].boxes.xywh.cpu()
                track_ids = results[0].boxes.id.int().cpu().tolist()
                annotated_frame = results[0].plot()
                for box, track_id in zip(boxes, track_ids):
                    x, y, w, h = box
                    track = track_history[track_id]
                    track.append((float(x), float(y)))
                    if len(track) > 30:
                        track.pop(0)
                    points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                    cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)
                cv2.imshow("YOLO11 Tracking", annotated_frame)
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
        

        
        # Para cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

                #Etiqueta de seguimiento
                if box.id is not None:
                    track_id = str(int(box.id[0].tolist()))
                else:
                    track_id = ''
                
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100
                print("Confianza --->",confidence)

                # Clase
                cls = int(box.cls[0])
                print("Clase -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0

                # Dibuja el contenedor y clase
                cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(img, track_id + ' ' + classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

        # Muestra fotograma
        cv2.imshow('Vid', img)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()




0: 480x640 1 person, 58.1ms
Speed: 9.2ms preprocess, 58.1ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
Confianza ---> 0.9
Clase --> person

0: 480x640 1 person, 83.8ms
Speed: 1.2ms preprocess, 83.8ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)
Confianza ---> 0.73
Clase --> person

0: 480x640 1 person, 63.2ms
Speed: 2.0ms preprocess, 63.2ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)
Confianza ---> 0.74
Clase --> person

0: 480x640 1 person, 52.0ms
Speed: 1.3ms preprocess, 52.0ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)
Confianza ---> 0.76
Clase --> person

0: 480x640 1 person, 68.0ms
Speed: 1.4ms preprocess, 68.0ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)
Confianza ---> 0.68
Clase --> person

0: 480x640 1 person, 60.3ms
Speed: 1.3ms preprocess, 60.3ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)
Confianza ---> 0.76
Clase --> person

0: 480x640 1 person, 5

Intregración con seguimiento (tracking)
!!!!!!!!!Nota: he tenido que bajar a la versión de python 3.9.5 e instalar lap con pip install lap

In [8]:
# Carga del modelo
#model = YOLO('yolo11n.pt') #Contenedores
#model = YOLO('yolov11n-seg.pt') #Máscaras
model = YOLO('yolo11n-pose.pt')  #Pose

#Para un vídeo 
filename = "TGC23_PdH_C0056cut.mp4"
results = model.track(source=filename, show=True)  # BoT-SORT tracker (por defecto)
#results = model.track(source=filename, show=True, tracker="bytetrack.yaml")  # ByteTrack tracker

cv2.destroyAllWindows()


WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/375) c:\Users\kevin\Ingeniera informtica\Cuarto\VC\P4\git\otsedom.github.io\VC\P4\TGC23_PdH_C0056cut.mp4: 384x640 (no detections), 58.1ms
video 1/1 (frame 2/375) c:\Users\kevin\Ingeniera informtica\Cuarto\VC\P4\git\otsedom.github.io\VC\P4\TGC23_PdH_C0056cut.mp4: 384x640 (no detections), 54.8ms
video 1/1 (frame 3/375) c:\Users\kevin\Ingeniera informtica\Cuarto\VC\P4\git\otsedom.github.io\VC\P4\TGC23_PdH_C0056cut.mp4: 384x640 (no detectio

In [3]:
from collections import defaultdict
import numpy as np
import cv2  
import math
from ultralytics import YOLO
import time

# Carga del modelo, descarga en disco si no está presente en la carpeta
model = YOLO('yolo11n.pt') #Contenedores
model_2 = YOLO('TGC_RBNW/runs/detect/train2/weights/best.pt')

# Etiqueta de las distintas clases
classNames = ["person", "bicycle", "car", "", "", "bus"]


# Captura desde la webcam
vid = cv2.VideoCapture("C0142.MP4")
track_history = defaultdict(lambda: [])
  
while(True):      
    # fotograma a fotograma
    ret, img = vid.read()
  
    # si hay imagen válida
    if ret:  
        # Seguimiento, con persistencia entre fotogramas
        results = model.track(img, persist=True, classes = [0,1,2,5])

        if 0:
            if results is not None:
                boxes = results[0].boxes.xywh.cpu()
                track_ids = results[0].boxes.id.int().cpu().tolist()
                annotated_frame = results[0].plot()
                for box, track_id in zip(boxes, track_ids):
                    x, y, w, h = box
                    track = track_history[track_id]
                    track.append((float(x), float(y)))
                    if len(track) > 30:
                        track.pop(0)
                    points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                    cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)
                cv2.imshow("YOLO11 Tracking", annotated_frame)
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
        

        
        # Para cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

                #Etiqueta de seguimiento
                if box.id is not None:
                    track_id = str(int(box.id[0].tolist()))
                else:
                    track_id = ''
                
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100

                # Clase
                cls = int(box.cls[0])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0

                # Dibuja el contenedor y clase
                cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(img, track_id + ' ' + classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)
                if cls == 2 or cls == 5:
                    plates = model_2(img[y1:y2, x1:x2], show=False)
                    for plate in plates:
                        for plate_box in plate.boxes:
                            x1_plate, y1_plate, x2_plate, y2_plate = plate_box.xyxy[0]
                            x1_plate, y1_plate, x2_plate, y2_plate = int(x1_plate), int(y1_plate), int(x2_plate), int(y2_plate) # convert to int values
                            cv2.rectangle(img, (x1 + x1_plate, y1 + y1_plate), (x2 + x2_plate, y2 + y2_plate), (R, G, B), 3)
                    

        # Muestra fotograma
        cv2.imshow('Vid', img)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()




0: 384x640 4 cars, 1 bus, 58.8ms
Speed: 19.5ms preprocess, 58.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 352x416 (no detections), 43.1ms
Speed: 1.1ms preprocess, 43.1ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 416)

0: 320x416 (no detections), 38.5ms
Speed: 0.9ms preprocess, 38.5ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 416)

0: 288x416 (no detections), 34.8ms
Speed: 0.8ms preprocess, 34.8ms inference, 0.3ms postprocess per image at shape (1, 3, 288, 416)

0: 288x416 (no detections), 34.4ms
Speed: 0.8ms preprocess, 34.4ms inference, 0.3ms postprocess per image at shape (1, 3, 288, 416)

0: 320x416 (no detections), 39.9ms
Speed: 1.0ms preprocess, 39.9ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 416)

0: 384x640 4 cars, 1 bus, 52.9ms
Speed: 1.7ms preprocess, 52.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 352x416 (no detections), 45.3ms
Speed: 1.1ms preprocess, 45.3ms infe

In [ ]:
from collections import defaultdict
import cv2  
from ultralytics import YOLO

# Carga del modelo, descarga en disco si no está presente en la carpeta
model = YOLO('yolo11n.pt') #Contenedores
model_2 = YOLO('TGC_RBNW/runs/detect/train2/weights/best.pt')

# Etiqueta de las distintas clases
classNames = ["person", "bicycle", "car", "", "", "bus"]

img = cv2.imread("captura.png")

results = model.track(img, persist=True, classes = [0,1,2,5])
for r in results:
    for box in boxes:
        plates = model_2(img[])


0: 352x640 10 cars, 67.9ms
Speed: 20.9ms preprocess, 67.9ms inference, 2.3ms postprocess per image at shape (1, 3, 352, 640)
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 5